# Lesson 12: Otsu's Binarization — Deep Dive

Otsu's method automatically selects the optimal threshold by analyzing the **histogram** of the image.  
It minimizes **intra-class variance** (equivalently, maximizes **between-class variance**) of background and foreground.

## How it works:
1. Try every possible threshold T from 0 to 255
2. For each T, split pixels into two groups: background (< T) and foreground (≥ T)
3. Compute the weighted variance of each group
4. Choose the T that **minimizes** the total intra-class variance

Best suited for **bimodal histograms** (two clear peaks = two dominant intensity groups)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image as grayscale
img = cv2.imread('images.jpg', cv2.IMREAD_GRAYSCALE)
if img is None:
    img = cv2.imread('sealion_hero.png', cv2.IMREAD_GRAYSCALE)

print(f'Image: {img.shape}, range: [{img.min()}, {img.max()}]')

## Part 1: Otsu's Method vs. Global Threshold (T=127)

Comparing fixed threshold vs automatically selected threshold.

In [ ]:
# Global threshold (manual)
_, global_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# Otsu's automatic threshold
otsu_T, otsu_binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f"Fixed threshold:  T = 127")
print(f"Otsu's threshold: T = {int(otsu_T)} (automatically found)")

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img, cmap='gray');             axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(global_binary, cmap='gray');   axes[1].set_title('Global T=127'); axes[1].axis('off')
axes[2].imshow(otsu_binary, cmap='gray');     axes[2].set_title(f"Otsu T={int(otsu_T)}"); axes[2].axis('off')
plt.tight_layout()
plt.show()

## Part 2: Histogram Visualization — Why Otsu Works

The histogram shows the intensity distribution. Otsu finds the "valley" between peaks.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].imshow(img, cmap='gray')
axes[0].set_title('Grayscale Image')
axes[0].axis('off')

axes[1].hist(img.ravel(), bins=256, range=[0, 256], color='steelblue', alpha=0.7)
axes[1].axvline(x=127, color='orange', linestyle='--', linewidth=2, label='T=127 (manual)')
axes[1].axvline(x=otsu_T, color='red', linestyle='-', linewidth=2, label=f'T={int(otsu_T)} (Otsu)')
axes[1].set_title('Pixel Intensity Histogram')
axes[1].set_xlabel('Pixel Intensity (0-255)')
axes[1].set_ylabel('Pixel Count')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: Otsu + Gaussian Blur Pre-processing

**Problem:** Images with noise confuse Otsu — the histogram has many small peaks instead of two clear ones.  
**Solution:** Apply Gaussian blur **before** Otsu to smooth out noise and get a cleaner bimodal histogram.

In [ ]:
# Add Gaussian noise to simulate noisy image
noise = np.random.normal(0, 25, img.shape).astype(np.int16)
noisy = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

# Otsu on noisy image (without pre-processing)
otsu_T_noisy, otsu_noisy = cv2.threshold(noisy, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Gaussian blur first, then Otsu
blurred = cv2.GaussianBlur(noisy, (5, 5), 0)
otsu_T_blur, otsu_blur = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f"Otsu on noisy image:          T = {int(otsu_T_noisy)}")
print(f"Otsu after Gaussian blur:     T = {int(otsu_T_blur)}")

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img, cmap='gray');         axes[0].set_title('Original'); axes[0].axis('off')
axes[1].imshow(noisy, cmap='gray');       axes[1].set_title('Noisy'); axes[1].axis('off')
axes[2].imshow(otsu_noisy, cmap='gray');  axes[2].set_title(f'Otsu on noisy\n(T={int(otsu_T_noisy)})'); axes[2].axis('off')
axes[3].imshow(otsu_blur, cmap='gray');   axes[3].set_title(f'Blur + Otsu\n(T={int(otsu_T_blur)})'); axes[3].axis('off')

plt.suptitle('Effect of Gaussian pre-processing on Otsu thresholding', fontsize=12)
plt.tight_layout()
plt.show()

## Part 4: Manually Implementing Otsu's Algorithm

Let's implement Otsu's method from scratch to understand the math behind it.

In [ ]:
def otsu_threshold(gray_img):
    """Compute Otsu's optimal threshold manually."""
    hist, bins = np.histogram(gray_img.ravel(), bins=256, range=[0, 256])
    total_pixels = gray_img.size
    
    best_T = 0
    best_variance = 0

    for T in range(1, 256):
        # Background class (pixels < T)
        bg_pixels = hist[:T].sum()
        fg_pixels = hist[T:].sum()

        if bg_pixels == 0 or fg_pixels == 0:
            continue

        w_bg = bg_pixels / total_pixels
        w_fg = fg_pixels / total_pixels

        mu_bg = np.dot(np.arange(T), hist[:T]) / bg_pixels
        mu_fg = np.dot(np.arange(T, 256), hist[T:]) / fg_pixels

        # Between-class variance
        variance = w_bg * w_fg * (mu_bg - mu_fg) ** 2

        if variance > best_variance:
            best_variance = variance
            best_T = T

    return best_T

my_T = otsu_threshold(img)
cv_T, _ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print(f'My Otsu threshold:     T = {my_T}')
print(f"OpenCV Otsu threshold: T = {int(cv_T)}")
print(f"Match: {my_T == int(cv_T)}")

## Summary

- Otsu's method is **automatic** — no need to manually choose T
- It works best when the histogram is **bimodal** (two peaks: background + foreground)
- For noisy images: **apply Gaussian blur first**, then run Otsu
- The algorithm maximizes **between-class variance** to find the optimal split point